In [1]:
# 必要なモジュールをインポート
import os
from dotenv import load_dotenv
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.llms.openai import OpenAI

# 環境変数の取得
load_dotenv("../.env")
os.environ["OPENAI_API_KEY"] = os.getenv("API_KEY")

# モデル名
MODEL_NAME = "gpt-4o-mini"

In [2]:
# PDFドキュメントの読込
documents = SimpleDirectoryReader("./data/pdf").load_data()

In [ ]:
from llama_index.core.text_splitter import SentenceSplitter
from llama_index.core import Settings
import tiktoken

# NodeParserの作成
node_parser = SentenceSplitter(
    separator="。",
    chunk_size=256,
    chunk_overlap=16,
    tokenizer=tiktoken.encoding_for_model(MODEL_NAME).encode)

# 言語モデルの指定
llm = OpenAI(model=MODEL_NAME, temperature=0.3)

# 設定に反映
Settings.llm = llm
Settings.node_parser = node_parser

# Indexの構築
index = VectorStoreIndex.from_documents(documents)

Retrying llama_index.embeddings.openai.base.OpenAIEmbedding._get_text_embeddings.<locals>._retryable_get_embeddings in 1.0 seconds as it raised RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}.


In [ ]:
# ストレージに保存
index.storage_context.persist("./storage01")

In [ ]:
from llama_index.core import StorageContext, load_index_from_storage

# ストレージコンテキストの作成
storage_context = StorageContext.from_defaults(persist_dir="./storage01")
# Indexのロード
index = load_index_from_storage(storage_context)

In [ ]:
sys_prompt_str = """
事前知識ではなく、常に提供されたコンテキスト情報を使用して質問に回答してください。
回答内でコンテキストを直接参照しないでください。
「コンテキストに基づいて」や「コンテキスト情報は」、またはそれに類するような記述は避けてください。
"""

qa_prompt_str = """
コンテキスト情報は以下の通りです。
---------------------
{context_str}
---------------------
事前知識ではなくコンテキスト情報を使用して、質問に回答してください。
質問: {query_str}
回答:"""

refine_prompt_str = """
元の回答を（必要な場合のみ）以下のコンテキストで改良する機会があります。
---------------------
{context_msg}
---------------------
新しいコンテキストが与えられた場合、元の回答を改良して、質問{query_str}に適切に回答します。
コンテキストが役に立たない場合は、元の回答を再度出力します。
元の回答: {existing_answer}"""

In [ ]:
from llama_index.core.llms import ChatMessage, MessageRole
from llama_index.core import ChatPromptTemplate

# テキストQAテンプレートの作成
chat_text_qa_msgs = [
    ChatMessage(role=MessageRole.SYSTEM, content=sys_prompt_str),
    ChatMessage(role=MessageRole.USER, content=qa_prompt_str),
]
text_qa_template = ChatPromptTemplate(chat_text_qa_msgs)

# リファインテンプレートの作成
chat_refine_msgs = [
    ChatMessage(role=MessageRole.SYSTEM, content=sys_prompt_str),
    ChatMessage(role=MessageRole.USER, content=refine_prompt_str),
]
refine_template = ChatPromptTemplate(chat_refine_msgs)

In [ ]:
# Chat Engineの作成
chat_engine = index.as_chat_engine(
    chat_mode="openai",
    llm=llm,
    similarity_top_k=3,
    text_qa_template=text_qa_template,
    refine_template=refine_template
)

In [ ]:
# 質問：1回目
response = chat_engine.chat("公共交通機関の交通費の上限は？")
for token in response.response_gen:
    print(token, end="")

In [ ]:
# 引用元を表示
for source in response.sources:
    for source_node in source.raw_output.source_nodes:
        print("ファイル名：", source_node.metadata["file_name"])
        print("関連度スコア:", source_node.score)
        print("テキスト：")
        print(source_node.node.text)
        print("-" * 50)  # 区切り線

In [ ]:
# 質問：2回目
response = chat_engine.stream_chat("交通費以外の手当にはどのようなものがありますか？")

for token in response.response_gen:
    print(token, end="")

In [ ]:
# 引用元を表示
for source in response.sources:
    for source_node in source.raw_output.source_nodes:
        print("ファイル名：", source_node.metadata["file_name"])
        print("関連度スコア:", source_node.score)
        print("テキスト：")
        print(source_node.node.text)
        print("-" * 50)  # 区切り線